# Exercise 5 - Electricity Market Properties

## Imports

In [6]:
import gurobipy as gp
from gurobipy import GRB

### Statement

In [7]:
generators = ['W1', 'G1', 'G2']
loads = ['D1', 'D2']

# Costs and capacities for generators
cost_beta = {'W1': 0, 'G1': 30, 'G2': 35}
gen_capacity = {'W1': 80, 'G1': 80, 'G2': 80}

# Utility and capacities for loads
utility = {'D1': 40, 'D2': 20}
load_capacity = {'D1': 100, 'D2': 50}

### Resolution

In [8]:
model = gp.Model('Electricity_Market')

# Add variables for power generation and load
pg = {g: model.addVar(lb=0, ub=gen_capacity[g], name='Power_Generation_in_{0}'.format(g)) for g in generators}
ld = {d: model.addVar(lb=0, ub=load_capacity[d], name='Load_for_{0}'.format(d)) for d in loads}

model.update()
print("Number of variables:", model.NumVars)

# Objective function: Maximize social welfare = utility from consumers - cost of generation
obj = (gp.quicksum(utility[d] * ld[d] for d in loads) -
       gp.quicksum(cost_beta[g] * pg[g] for g in generators))

model.setObjective(obj, GRB.MAXIMIZE)

# Balance constraint: Total generation must meet total demand
balance_constraint = model.addLConstr(
    gp.quicksum(pg[g] for g in generators),
    GRB.EQUAL,
    gp.quicksum(ld[d] for d in loads),
    name='Balance_Equation'
)

# Add generator constraints (implicit in variable bounds)
max_production_constraint = {g: model.addConstr(pg[g] <= gen_capacity[g], name='Generator_{0}_Max_Production'.format(g)) for g in generators}

# Add minimum production constraints (implicit as non-negativity)
min_production_constraint = {g: model.addConstr(pg[g] >= 0, name='Generator_{0}_Min_Production'.format(g)) for g in generators}

# Solve the model
model.optimize()

# Retrieve results
if model.status == GRB.OPTIMAL:
    print("\n-------------------   RESULTS  -------------------")
    optimal_objective = model.ObjVal  # Save objective value of social welfare at optimality
    constraints = model.getConstrs()  # List of constraints
    optimal_sensitivities = [constraints[c].Pi for c in range(len(constraints))]  # Save value of dual variables for constraints

    print(f"Optimal social welfare: {optimal_objective:.2f}")
    
    print("\nOptimal generation levels:")
    for g in generators:
        print(f"Generator {g}: {pg[g].x:.2f} MWh")
    
    print("\nOptimal load levels:")
    for d in loads:
        print(f"Load {d}: {ld[d].x:.2f} MWh")

    # Dual values for the balance equation
    print(f'\nOptimal dual of balance equation: {balance_constraint.Pi:.3f}')
    print(f'Slack in balance equation: {balance_constraint.Slack:.3f}')

    # Dual values for max production constraints
    print("\nOptimal duals of max production constraints:")
    for g in generators:
        print(f"Dual U_{g}: {max_production_constraint[g].Pi:.3f}")
        print(f"Slack U_{g}: {max_production_constraint[g].Slack:.3f}")

    # Dual values for min production constraints
    print("\nOptimal duals of min production constraints:")
    for g in generators:
        print(f"Dual u_{g}: {min_production_constraint[g].Pi:.3f}")
        print(f"Slack u_{g}: {min_production_constraint[g].Slack:.3f}")

else:
    raise RuntimeError(f"Optimization of {model.ModelName} was not successful")

Number of variables: 5
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10700T CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 7 rows, 5 columns and 11 nonzeros
Model fingerprint: 0x907dd089
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 4e+01]
  Bounds range     [5e+01, 1e+02]
  RHS range        [8e+01, 8e+01]
Presolve removed 6 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 5 columns, 5 nonzeros
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+03   8.750000e+00   0.000000e+00      0s
       1    3.4000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.400000000e+03

-------------------   RESULTS  -------------------
Optimal social welfare: 3400.00

Optimal gene